# Baselines

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from baselines import get_baseline
from baselines import multi_class_roc_auc, multi_class_spearman
from baselines import empirical_dist
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
d = pd.read_csv('../../data/annotations/baseline/annotations.tsv', sep='\t')
d.index = d.rev_id

In [3]:
d_b = d.query("sample=='blocked'")
d_r = d.query("sample=='random'")

In [4]:
iters = 10
K = 20
tasks = ['attack', 'recipient', 'aggression']
metrics = {'ROC': multi_class_roc_auc, 'spearman':multi_class_spearman}
ds = {'blocked': d_b, 'random':d_r}

In [5]:
def get_all_baselines(K, tasks, metrics, ds, iters, pairs, n_jobs = 8):
    df_results = []
    results = {}
    
    for task in tasks:
        results[task] = {}
        for d_name, d in ds.items():
            results[task][d_name] = {}
            for metric_name, metric in metrics.items():
                results[task][d_name][metric_name] = {pair:[] for pair in pairs}
                l = d[task]
                for i in range(iters):
                    pms = get_baseline(l, K, empirical_dist, metric, pairs, n_jobs=n_jobs)
                    for pair, m in pms.items():
                        results[task][d_name][metric_name][pair].append(m)
                        
                        
                        
    for task in tasks:
        print('Task: ', task)
        for d_name, d in ds.items():
            print('\tData: ', d_name)
            for metric_name, metric in metrics.items():
                print('\t\tMetric: ', metric_name)
                for pair in pairs:
                    a = np.array(results[task][d_name][metric_name][pair])
                    print('\t\t\t(%d, %d): %0.3f (%0.3f)' % (pair[0], pair[1], a.mean(), a.std()))
                    df_results.append({'task':task,
                                       'data_type': d_name,
                                       'metric': metric_name,
                                       'n_pred': pair[0],
                                       'n_gold': pair[1],
                                       'score': a.mean(),
                                       'error': a.std(),
                                          })
    pd.DataFrame(df_results).to_csv('baselines.csv', index=False)

In [6]:
F = int(K/2)
pairs = list(zip(range(1, F+1), range(1, F+1))) + list(zip(range(1, F+1), [F]*F))

In [7]:
get_all_baselines(K, tasks, metrics, ds, iters, pairs, n_jobs=4)

Task:  attack
	Data:  random
		Metric:  spearman
			(1, 1): 0.185 (0.031)
			(2, 2): 0.251 (0.022)
			(3, 3): 0.269 (0.015)
			(4, 4): 0.291 (0.019)
			(5, 5): 0.319 (0.017)
			(6, 6): 0.322 (0.012)
			(7, 7): 0.323 (0.015)
			(8, 8): 0.335 (0.014)
			(9, 9): 0.339 (0.009)
			(10, 10): 0.344 (0.010)
			(1, 10): 0.225 (0.014)
			(2, 10): 0.283 (0.011)
			(3, 10): 0.290 (0.011)
			(4, 10): 0.317 (0.017)
			(5, 10): 0.316 (0.011)
			(6, 10): 0.327 (0.012)
			(7, 10): 0.336 (0.011)
			(8, 10): 0.341 (0.010)
			(9, 10): 0.344 (0.006)
			(10, 10): 0.344 (0.010)
		Metric:  ROC
			(1, 1): 0.597 (0.009)
			(2, 2): 0.812 (0.035)
			(3, 3): 0.825 (0.025)
			(4, 4): 0.933 (0.021)
			(5, 5): 0.925 (0.018)
			(6, 6): 0.957 (0.020)
			(7, 7): 0.963 (0.012)
			(8, 8): 0.989 (0.008)
			(9, 9): 0.981 (0.009)
			(10, 10): 0.983 (0.014)
			(1, 10): 0.733 (0.019)
			(2, 10): 0.943 (0.019)
			(3, 10): 0.914 (0.023)
			(4, 10): 0.966 (0.024)
			(5, 10): 0.956 (0.015)
			(6, 10): 0.980 (0.013)
			(7, 10): 0.9

In [8]:
#get_all_baselines(4, ['aggression',], {'ROC': multi_class_roc_auc}, {'blocked':ds['blocked']}, 2, [(1,3)])